In [1]:
import time
import spacy
import lftk
import pandas as pd
from datasets import load_dataset

In [3]:
# load a trained pipeline of your choice from spacy
nlp = spacy.load("en_core_web_sm")

# readability formula scores - we can always add more general linguistic features
metrics = lftk.search_features(family="readformula", return_format = "list_key")

In [14]:
# data: generated summaries, gold: human-written summaries
data = pd.read_csv('../output/baseline_test.csv', usecols=['summary_generated'])
gold = load_dataset("FiscalNote/billsum")["test"].to_pandas()

In [4]:
# returns a dictionary of readability scores from LFTK
def readability_score_single(text):
    doc = nlp(text)
    LFTK = lftk.Extractor(docs = doc)
    features = LFTK.extract(features = metrics)
    return features

def readability_eval(df, column):
    start_time = time.time()
    df["scores"] = df[column].apply(readability_score_single)
    print(f"{time.time() - start_time} sec to run")
    return

In [9]:
# replaces the dictionary column with exploded keys as new columns
def convert_cols(df, column):
    return df.join(
        pd.json_normalize(
            df[column]
        )
    ).drop(
        column,
        axis='columns'
    )

In [6]:
readability_eval(data, "summary_generated")
readability_eval(gold, "summary")

82.2096209526062 sec to run
103.58868312835693 sec to run


In [12]:
# Only run once!
# gen = convert_cols(data, "scores")
# gold = convert_cols(gold, "scores")

In [25]:
gold.drop(columns=['text', 'title'], inplace=True)

,summary,fkre,fkgl,fogi,smog,cole,auto
0,Amends the Water Resources Development Act of ...,-268.681,128.741,137.320,35.676,17.981,162.705
1,Federal Forage Fee Act of 1993 - Subjects graz...,43.716,12.876,20.504,9.033,13.847,14.849
2,. Merchant Marine of World War II Congression...,64.618,9.836,13.000,6.387,9.671,11.277
3,Small Business Modernization Act of 2004 - Ame...,22.415,20.070,27.500,9.895,15.727,24.416
4,Fair Access to Investment Research Act of 2016...,17.456,20.811,26.826,12.774,14.560,23.579
...,...,...,...,...,...,...,...
3264,Public Servant Priority Placement Act of 1995 ...,-2.963,26.094,34.400,5.713,15.478,29.098
3265,Sportsmanship in Hunting Act of 2008 - Amends ...,21.447,18.963,26.571,9.474,13.510,20.226
3266,Helping College Students Cross the Finish Line...,19.509,22.314,27.230,11.136,13.695,26.396
3267,Makes proceeds from such conveyances available...,0.151,20.194,26.914,13.993,19.683,21.839


In [23]:
gen.to_csv("gen_all_readability_scores.csv", index=False, escapechar='\\')
gold.to_csv("gold_all_readability_scores.csv", index=False, escapechar='\\')

In [18]:
gold.loc[gold.summary.str.contains("Children's Bicycle Helmet Safety Act of 1993")].text.tolist()

["SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Children's Bicycle Helmet Safety Act \nof 1993''.\n\nSEC. 2. FINDINGS.\n\n    The Congress finds that--\n            (1) 90 million Americans ride bicycles and 20 million ride \n        a bicycle more than once a week;\n            (2) between 1984 and 1988, 2,985 bicyclists in the United \n        States died from head injuries and 905,752 suffered head \n        injuries that were treated in hospital emergency rooms;\n            (3) 41 percent of bicycle-related head injury deaths and 76 \n        percent of bicycle-related head injuries occurred among \n        American children under age 15;\n            (4) deaths and injuries from bicycle accidents cost society \n        $7.6 billion annually; and a child suffering from a head \n        injury, on average, will cost society $4.5 million over the \n        child's lifetime;\n            (5) universal use of bicycle helmets in the United States \n        would have pr

In [20]:
# gold.describe().to_csv("gold_describe.csv")
gold.describe().compare(gen.describe(), keep_equal=True, align_axis=1, result_names=("gold", "generated"))

# aggr.to_csv("baseline_gold_compare.csv", index=False)

fkre                    fkgl                   fogi              \
            gold   generated        gold  generated        gold   generated   
mean   12.950785    5.292479   22.548614  26.011575   28.701746   32.256271   
std    28.561424   39.147077   10.030598  14.147657   10.336453   14.545976   
min  -333.371000 -197.615000    6.311000   6.217000   10.150000    8.969000   
25%     2.670000   -8.085000   16.792000  17.666000   22.815000   23.667000   
50%    17.009000   12.846000   20.577000  22.472000   26.733000   28.619000   
75%    29.321000   29.003000   25.451000  29.742000   31.892000   36.000000   
max    80.098000   81.956000  154.161000  97.404000  163.263000  105.737000   

           smog                  cole                   auto              
           gold  generated       gold  generated        gold   generated  
mean  11.587442  12.407292  15.162678  14.809797   26.354612   30.819341  
std    3.764673   4.721686   2.216489   2.464433   12.738404   17.997384  
min    0.000000   0.000000   7.802000   6.261000    5.720000    6.407000  
25%    9.329000   9.329000  13.660000  13.118000   19.089000   20.248000  
50%   11.220000  11.892000  15.045000  14.684000   23.714000   26.103000  
75%   13.398000  14.750000  16.556000  16.320000   29.975000   35.103000  
max   35.676000  36.579000  25.251000  24.759000  194.066000  122.723000

In [15]:
# gen.describe().to_csv("gen_describe.csv")
gen.describe()

,fkre,fkgl,fogi,smog,cole,auto
count,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000
mean,5.292479,26.011575,32.256271,12.407292,14.809797,30.819341
std,39.147077,14.147657,14.545976,4.721686,2.464433,17.997384
min,-197.615000,6.217000,8.969000,0.000000,6.261000,6.407000
25%,-8.085000,17.666000,23.667000,9.329000,13.118000,20.248000
50%,12.846000,22.472000,28.619000,11.892000,14.684000,26.103000
75%,29.003000,29.742000,36.000000,14.750000,16.320000,35.103000
max,81.956000,97.404000,105.737000,36.579000,24.759000,122.723000


In [2]:
gold_scores = pd.read_csv("gold_all_readability_scores.csv")
gen_scores = pd.read_csv("gen_all_readability_scores.csv")

In [25]:
flist = lftk.search_features(family="worddiff", return_format="list_key")
flist.append("t_n_ent_law")
flist

['t_kup', 't_bry', 't_subtlex_us_zipf', 't_n_ent_law']

In [32]:
# returns a dictionary of readability scores from LFTK
def feature_single(text, feat):
    doc = nlp(text)
    LFTK = lftk.Extractor(docs = doc)
    features = LFTK.extract(features = [feat])
    return features[feat]

def feature_eval(df, column):
    for f in flist:
        print(f)
        start_time = time.time()
        df[f] = df[column].apply(lambda r: feature_single(r, f))
        print(f"{time.time() - start_time} sec to run")
    return

In [33]:
feature_eval(gen_scores, "summary_generated")
feature_eval(gold_scores, "summary")

t_kup
997.7767179012299 sec to run
t_bry
1341.2437300682068 sec to run
t_subtlex_us_zipf
2147.247776031494 sec to run
t_n_ent_law
103.05808711051941 sec to run
t_kup
4688.640254974365 sec to run
t_bry
3580.801120996475 sec to run
t_subtlex_us_zipf
2131.8419280052185 sec to run
t_n_ent_law
100.15771007537842 sec to run


In [36]:
for c in gen_scores.columns:
    if "t_" in c:
        gen_final[c] = gen_scores[c]
        gold_final[c] = gold_scores[c]

In [10]:
gen_final = convert_cols(gen_scores, "features")
gold_final = convert_cols(gold_scores, "features")

In [12]:
gen_final.to_csv("gen_readformula_wordsent_worddiff_features.csv")
gold_final.to_csv("gold_readability_wordsent_features.csv")

In [39]:
gen_final[[c for c in gen_final.columns if c in flist]].describe()

,t_kup,t_bry,t_subtlex_us_zipf,t_n_ent_law
count,3269.000000,3269.000000,3269.000000,3269.000000
mean,725.649061,581.660470,694.295961,0.222086
std,379.991266,307.656867,339.627031,0.541080
min,44.990000,31.536000,102.607000,0.000000
25%,378.490000,302.351000,384.765000,0.000000
50%,707.880000,561.962000,683.621000,0.000000
75%,1081.000000,869.513000,1035.688000,0.000000
max,1517.120000,1274.352000,1317.443000,5.000000


In [40]:
gold_final[[c for c in gold_final.columns if c in flist]].describe()

,t_kup,t_bry,t_subtlex_us_zipf,t_n_ent_law
count,3269.000000,3269.000000,3269.000000,3269.000000
mean,958.431490,764.343065,882.934860,0.298256
std,650.269915,523.754160,577.809586,0.686511
min,25.080000,29.500000,48.670000,0.000000
25%,476.330000,372.024000,452.264000,0.000000
50%,829.150000,658.951000,763.686000,0.000000
75%,1289.520000,1029.839000,1188.576000,0.000000
max,4478.610000,3642.328000,3995.888000,7.000000


In [44]:
gen_final.to_csv("gen_lftk.csv")
gold_final.to_csv("gold_lftk.csv")